# Tutorial

---

## Table of Contents 

* [1. Introduction](#introduction)
    * [1.1. Project Description](#project-description)
    * [1.2. Packages](#data-sources)
* [2. Extract, transform and load](#extract-transform-and-load)
    * [2.1. Setting configuration, directory and filepaths](#setting-configuration-directory-and-filepaths)
    * [2.2. Creating the isoforms data set](#creating-the-isoforms-data-set)
* [3. TRIFID model training](#trifid-model-training)
    * [3.1. Setting seed and training mode](#setting-seed-and-training-mode)
    * [3.2. Evaluating the selected model](#evaluating-the-selected-model)
* [4. Predicting new isoforms functionality](#predicting-new-isoforms-functionality)
    * [4.1. Predicting GENCODE 27 (human) isoforms data set](#predicting-gencode-27-human-isoforms-data-set)
    * [4.2. Predicting GENCODE 33 (human) isoforms data set](#predicting-gencode-33-human-isoforms-data-set)
* [5. Quantifying TRIFID predictions over the whole genome](#quantifying-trifid-predictions-over-the-whole-genome)
    * [5.1. Crossing with APPRIS](#crossing-with-appris)
        * [5.1.1. Non redundant data set](#non-redundant-data-set)
        * [5.1.2. Complete data set](#complete-data-set)
    * [5.2. Crossing with PULSE](#crossing-with-pulse)
* [6. Interpreting our predictions](#interpret-the-predictions)
    * [6.1. Global interpretation](#global-interpretation)
    * [6.2. Local interpretations](#local-interpretations)
        * [6.2.1. Local interpretation for set of gene isoforms](#local-interpretation-for-set-of-gene-isoforms)
        * [6.2.2. Local interpretation for a single isoform](#local-interpretation-for-a-single-isoform)
* [7. Project contribution](#project-contribution)

---

# 1. Introduction <a class="anchor" id="introduction"></a>

## 1.1. Project Description <a class="anchor" id="project-description"></a>

Alternative Splicing (AS) of messenger RNA can generate a wide variety of mature RNA transcripts and this expression is confirmed by experimental transcript evidence. In theory these transcripts could generate protein isoforms with diverse cellular functions. However, while peptide evidence strongly supports a main protein isoform for the vast majority of coding genes [(1)](https://pubs.acs.org/doi/full/10.1021/pr501286b), it is not clear what proportion of these AS isoforms form stable functional proteins. In fact reliable proteomics experiments have found little evidence of alternative spliced proteins [(2)](https://www.sciencedirect.com/science/article/pii/S0968000416301189), so the number of stably folded/functional proteins produced by AS remains a mystery.

We have developed a computational method (`TRIFID`) for the classification of splice isoform functional importance. This algorithm was trained on reliable peptide evidence from proteomics analyses and classifies biologically important splice isoforms with high confidence. The algorithm ranks the most significant biological splice isoforms and we show that the highest scoring alternative exons are actually under selection pressure, unlike the vast majority of alternative exons. TRIFID can predict functional isoforms for any well-annotated eukaryotic species. The method will generate valuable insights into the cellular importance of alternative splicing.

## 1.2. Packages <a class="anchor" id="packages"></a>

In [ ]:
import os, pickle, sys
sys.path.append('../')
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

sys.path.append('../')
from trifid.data.loaders import Fasta
from trifid.data.feature_engineering import build_features, load_data
from trifid.utils.utils import *
from trifid.models.select import ModelSelection, Classifier
from trifid.models.interpret import TreeInterpretation

%load_ext autoreload
%autoreload 2

%load_ext watermark
%watermark -a 'Fernando Pozo' -u -n -t -z -g -p os,pickle,sys,warnings,pandas,numpy,sklearn,trifid

!pwd

---

# 2. Extract, transform and load <a class="anchor" id="extract-transform-and-load"></a>

## 2.1 Setting configuration, directory and filepaths <a class="anchor" id="setting-configuration-directory-and-filepaths"></a>

In [ ]:
TRIFID_DIR = '~/Projects/trifid'
CONFIG_PATH = os.path.join(TRIFID_DIR, 'config/config.yaml')
FEATURES_PATH = os.path.join(TRIFID_DIR, 'config/features.yaml')

In [ ]:
df_g27_sequences = Fasta(os.path.join(TRIFID_DIR, 'data', 'source', 'GRCh38', 'g27', 'appris_data.transl.fa.gz'), db='g').load
df_g27_sequences['transcript_id'] = df_g27_sequences['id'].str.split('|').str[1]
df_g27_sequences = df_g27_sequences[['transcript_id', 'sequence']]

In [ ]:
config = parse_yaml(CONFIG_PATH)
df_features = pd.DataFrame(parse_yaml(FEATURES_PATH))
features = df_features[~df_features['category'].str.contains('Identifier')]['feature'].values
ids = df_features[df_features['category'].str.contains('Identifier')]['feature'].values

In [ ]:
print(f"Annotation features: {df_features[df_features['category'].str.contains('Annotation')]['feature'].shape[0]} \n\
Evolution features: {df_features[df_features['category'].str.contains('Evolution')]['feature'].shape[0]} \n\
Expression features: {df_features[df_features['category'].str.contains('Expression')]['feature'].shape[0]} \n\
Splicing features: {df_features[df_features['category'].str.contains('Splicing')]['feature'].shape[0]} \n\
Structure features: {df_features[df_features['category'].str.contains('Structure')]['feature'].shape[0]}\n\n\
Total features: {df_features[~df_features['category'].str.contains('Identifier')]['feature'].shape[0]}")

In [ ]:
config['genomes']['GRCh38']['g27']

## 2.2 Creating the isoforms data set <a class="anchor" id="creating-the-isoforms-data-set"></a>

In [ ]:
create_dir(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'g27'))
df_g27 = load_data(config, assembly='GRCh38', release='g27')
df_g27 = build_features(df_g27)
# df_g27[df_features.feature.values].drop('sequence', axis=1).to_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'g27', 'trifid_db.tsv.gz'), index=None, sep='\t', compression='gzip')

---

# 3. TRIFID model training <a class="anchor" id="trifid-model-training"></a>

Random Forest (RF) was the method selected for TRIFID. This classifier is an ensemble-based learning algorithm containing a number of de-correlated decision trees. It uses multiple trees to compute majority votes in the final leaf nodes when making a prediction ([Breiman, 2001](https://link.springer.com/article/10.1023/A:1010933404324)). This learner is built around the idea of bagging, which generates bootstrap samples from the original data set for resampling with replacement in order to reduce variance ([Breiman, 1996](https://link.springer.com/article/10.1007%2FBF00058655); [Gordon et al., 1984](https://www.jstor.org/stable/2530946?origin=crossref&seq=1). At each node, a given number of input features are randomly chosen and the best split is calculated only within this subset. The individual trees are unpruned, and the last leaves do not contain more than the minimum size of terminal nodes of observations. It performs well on binary classification tasks due to their robust performance across a wide range of data sets. It is a suitable method for this task due to:

- RF has ability to handle categorical, boolean and continuous features. 
- RF does not require aggressive feature selection to reach adequate performance.
- RF can handle correlated features.
- RF can handle missing data. It is highly important for us, thinking on the ability of the model to be exported to different species or genome annotation databases.
- RF has only 2 or 3 parameters to be managed to avoid overfitting.
- RF releases high confidence interpretations with techniques like SHAP ([Lundberg et al., 2020](https://www.nature.com/articles/s42256-019-0138-9)). Moreover, SHAP brings back both global and local interpretation of TRIFID predictions.

Below, an hyperparameter tuning for best parameters of RF is shown. Performance metrics for single decission trees and K-Nearest Neighbors were also calculated in the model selection in order to set a standard line of performance.

## 3.1. Setting seed and training mode <a class="anchor" id="setting-seed-and-training-mode"></a>

In [ ]:
df_g27 = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'g27', 'trifid_db.tsv.gz'), sep='\t', compression='gzip')

In [ ]:
df_training_set_initial = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'model', 'training_set_initial.g27.tsv.gz'), sep='\t')
df_training_set = df_training_set_initial.copy()
df_training_set.loc[df_training_set['state'].str.contains('F'), 'label'] = 1
df_training_set.loc[df_training_set['state'].str.contains('U'), 'label'] = 0
df_training_set = df_training_set.loc[~df_training_set['label'].isnull()]
df_training_set = df_training_set.loc[df_training_set['added'].str.contains('v1|r|v3')].drop(['added', 'state', 'comment'], axis=1).reset_index(drop=True)
# df_training_set.to_csv(os.path.join(TRIFID_DIR, 'data', 'model', 'training_set_final.g27.tsv.gz'), sep='\t', compression='gzip', index=None)

In [ ]:
df_training_set.shape

In [ ]:
df_training_set.label.value_counts(normalize=True)*100

In [ ]:
SEED = 123
TRAINING_MODE = 'pretrained_model'

In [ ]:
if TRAINING_MODE == 'model_selection':
    ms = ModelSelection(df_training_set, 
                        features_col=df_training_set[features].columns,
                        target_col='label',
                        random_state=SEED)
    my_model = ms.get_best_model(outdir=os.path.join(TRIFID_DIR, 'models'))
    
elif TRAINING_MODE == 'custom_model': 
    my_model = RandomForestClassifier(
        min_samples_leaf=6, 
        n_estimators=400, 
        n_jobs=-1,
        random_state=123)    
elif TRAINING_MODE == 'pretrained_model':
    my_model = pickle.load(open(os.path.join(TRIFID_DIR, 'models', 'selected_model.pkl'), 'rb'))
    
model = Classifier(
    model = my_model,
    df = df_training_set, 
    features_col=df_training_set[features].columns,
    target_col='label',
    random_state=SEED)
model.save_model(outdir=os.path.join(TRIFID_DIR, 'models'))

## 3.2. Evaluating the selected model <a class="anchor" id="evaluating-the-selected-model"></a>

In [ ]:
model.model

In [ ]:
model.evaluate

In [ ]:
model.confusion_matrix

In [ ]:
print(model.classification_report)

In [ ]:
model.cross_validate

---

# 4. Predicting new isoforms functionality <a class="anchor" id="predicting-new-isoforms-functionality"></a>

## 4.1. Predicting GENCODE 27 (human) isoforms data set <a class="anchor" id="predicting-gencode-27-human-isoforms-data-set"></a>

In [ ]:
df_g27 = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'g27', 'trifid_db.tsv.gz'), sep='\t', compression='gzip')
df_g27_features = df_g27[features]
df_g27_predictions = df_g27[ids]
df_g27_predictions = generate_trifid_metrics(df_g27_predictions, df_g27_features, model)
df_g27_predictions = df_g27_predictions[['gene_id', 'gene_name', 'transcript_id', 'translation_id', 'flags',
                                         'ccdsid', 'appris', 'ann_type', 'length', 'trifid_score', 'norm_trifid_score', 
                                         ]]
df_g27_predictions.to_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'g27', 'trifid_predictions.tsv.gz'), index=None, sep='\t', compression='gzip')

In [ ]:
Statistics(df_g27_predictions).get_stats()

## 4.2 Predicting GENCODE 35 (human) isoforms data set <a class="anchor" id="predicting-gencode-35-human-isoforms-data-set"></a>

In [ ]:
df_g35 = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'g35', 'trifid_db.tsv.gz'), sep='\t', compression='gzip')
df_g35_features = df_g35[features]
df_g35_predictions = df_g35[ids]
df_g35_predictions = generate_trifid_metrics(df_g35_predictions, df_g35_features, model)
df_g35_predictions = df_g35_predictions[['gene_id', 'gene_name', 'transcript_id', 'translation_id', 'flags',
                                         'ccdsid', 'appris', 'ann_type', 'length', 'trifid_score', 'norm_trifid_score', 
                                         ]]
df_g35_predictions.to_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'g35', 'trifid_predictions.tsv.gz'), index=None, sep='\t', compression='gzip')

In [ ]:
Statistics(df_g35_predictions).get_stats()

## 4.3 Predicting RefSeq 109 (human) isoforms data set <a class="anchor" id="predicting-refseq109-human-isoforms-data-set"></a>

In [ ]:
features_no_rs109_ref = ['basic', 'level_1', 'level_2', 'level_3', 'nonsense_mediated_decay', 
                         'norm_PhyloCSF_Psi', 'norm_RelBranchLength', 'norm_ScorePerCodon', 
                        'PhyloCSF_Psi', 'RelBranchLength', 'ScorePerCodon', 'StartEnd_NF',
                        'tsl_1', 'tsl_2', 'tsl_3', 'tsl_4', 'tsl_5', 'tsl_6']

In [ ]:
df_rs109 = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'rs109', 'trifid_db.tsv.gz'), sep='\t', compression='gzip')

df_rs109.loc[df_rs109['ccdsid'].str.contains('CCDS'), 'CCDS'] = 1
df_rs109.loc[df_rs109['ccdsid'].str.contains('-'), 'CCDS'] = 0

df_rs109_predictions = df_rs109[ids]
df_rs109[features_no_rs109_ref] = df_rs109[features_no_rs109_ref].fillna(-1)
df_rs109[['corsair_alt', 'norm_corsair_alt']] = df_rs109[['corsair_alt', 'norm_corsair_alt']].fillna(0)
df_rs109_features = df_rs109[features]

df_rs109_predictions = generate_trifid_metrics(df_rs109_predictions, df_rs109_features, model)
df_rs109_predictions = df_rs109_predictions[['gene_id', 'gene_name', 'transcript_id', 'translation_id', 'flags',
                                         'ccdsid', 'appris', 'ann_type', 'length', 'trifid_score', 'norm_trifid_score', 
                                         ]]
df_rs109_predictions.to_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCh38', 'rs109', 'trifid_predictions.tsv.gz'), index=None, sep='\t', compression='gzip')

In [ ]:
Statistics(df_rs109_predictions).get_stats()

## 4.5. Predicting GENCODE 25 (mouse) isoforms data set <a class="anchor" id="predicting-gencode25-mouse-isoforms-data-set"></a>

In [ ]:
df_gm25 = pd.read_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCm38', 'g25', 'trifid_db.tsv.gz'), sep='\t', compression='gzip')
df_gm25_features = df_gm25[features]
df_gm25_predictions = df_gm25[ids]
df_gm25_predictions = generate_trifid_metrics(df_gm25_predictions, df_gm25_features, model)
df_gm25_predictions = df_gm25_predictions[['gene_id', 'gene_name', 'transcript_id', 'translation_id', 'flags',
                                         'ccdsid', 'appris', 'ann_type', 'length', 'trifid_score', 'norm_trifid_score', 
                                         ]]
df_gm25_predictions.to_csv(os.path.join(TRIFID_DIR, 'data', 'genomes', 'GRCm38', 'g25', 'trifid_predictions.tsv.gz'), index=None, sep='\t', compression='gzip')

In [ ]:
Statistics(df_gm25_predictions).get_stats()

---

# 5. Quantifying TRIFID predictions over the whole genome <a class="anchor" id="quantifying-trifid-predictions-over-the-whole-genome"></a>

## 5.1. Crossing with APPRIS <a class="anchor" id="crossing-with-appris"></a>

[Rodriguez JM, Rodriguez-Rivas J, Di Domenico T, Vázquez J, Valencia A, Tress ML. APPRIS 2017: principal isoforms for multiple gene sets. Nucleic Acids Res. 2018;46(D1):D213-D217.](doi:10.1093/nar/gkx997)

http://appris-tools.org

### 5.1.1. Non redundant dataset <a class="anchor" id="non-redundant-data-set"></a>

GENCODE v27 coding transcripts would produce a redundant set of proteins; more than a third of transcripts differ only in the 5’ and 3’ untranslated regions (UTR) or would produce translations that are fragments of other proteins. In order to generate a non-redundant set of protein isoforms for analysis by our model, we filtered out transcripts from the same gene that would produce identical proteins or that were tagged as being incomplete fragments with the same CDS (Coding DNA Sequence) as other coding transcripts.
 
For the analysis of the non-redundant set of isoforms generated from protein coding transcripts, we also filtered transcripts from genes that were not tagged as protein coding genes (such as immunoglobulin or T-cell receptor fragments) and also those transcripts tagged with labels such as “NMD” (nonsense mediated decay), “NSD” (non stop decay) or “Readthrough” that are highly unlikely to produce functional proteins.

In [ ]:
Statistics(df_g27_predictions)

### 5.1.2. Complete dataset <a class="anchor" id="complete-data-set"></a>

In [ ]:
Statistics(df_g27_predictions, nr=False).get_stats(cutoff=0.5, norm_double_check=True)

In [ ]:
Statistics(df_g34_predictions, nr=False).get_stats()

## 5.2. Crossing with PULSE <a class="anchor" id="crossing-with-pulse"></a>

[Hao Y, Colak R, Teyra J, et al. Semi-supervised Learning Predicts Approximately One Third of the Alternative Splicing Isoforms as Functional Proteins. Cell Rep. 2015;12(2):183-189.](doi:10.1016/j.celrep.2015.06.031)

In [ ]:
# !mkdir -p ../references/pulse/predictions/

In [ ]:
# !curl https://ars.els-cdn.com/content/image/1-s2.0-S2211124715006439-mmc3.xlsx -o ../references/pulse/predictions/1-s2.0-S2211124715006439-mmc3.xlsx

In [ ]:
df_pulse = pd.read_excel('../references/pulse/predictions/1-s2.0-S2211124715006439-mmc3.xlsx', skiprows=18)
df_pulse = df_pulse.rename(columns={'Sequence': 'sequence', 'Score': 'PULSE'})
df_pulse = pd.merge(
    df_pulse[['sequence', 'PULSE']], 
    df_g27_predictions[['transcript_id', 'appris', 'sequence', 'trifid_score']], 
    how='left', on='sequence')
df_pulse = df_pulse.loc[~df_pulse['transcript_id'].isnull()].reset_index(drop=True)
df_pulse = df_pulse.loc[~df_pulse['appris'].str.contains(
    'PRINCIPAL')].drop_duplicates('sequence').reset_index(drop=True)
df_pulse.head()

## 5.3. Final statistics

In [ ]:
def get_final_trifid_stats(df:list, version:str=27)->str:
    n_genes_g27 = df.gene_name.nunique()
    n_isoforms_g27 = df.shape[0]
    n_PRI_appris_g27 = df[df['appris'].str.contains('PRINCIPAL')].shape[0]
    n_ALT_appris_g27 = df[df['appris'].str.contains('ALTERNATIVE')].shape[0]
    n_alt_appris_g27 = df[~df['appris'].str.contains('PRINCIPAL')].shape[0]
    n_minor_appris_g27 = df[df['appris'].str.contains('MINOR')].shape[0]
    n_nr_isoforms_g27 = df.loc[df['ann_type'].str.contains('^Alternative$|^Principal$')].loc[df['flags'].str.contains('^protein_coding$')].shape[0]
    n_nr_pc_genes_g27 = df.loc[df['ann_type'].str.contains('^Alternative$|^Principal$')].loc[df['flags'].str.contains('^protein_coding$')].gene_name.nunique()

    msg=f'''IN GENCODE {version}, there are:
          {n_genes_g27} genes. 
          {n_isoforms_g27} isoforms.\n
          {n_PRI_appris_g27} APPRIS PRINCIPAL isoforms.
          {n_ALT_appris_g27} APPRIS ALTERNATIVE isoforms.
          {n_minor_appris_g27} APPRIS MINOR isoforms.
          {n_alt_appris_g27} APPRIS non PRINCIPAL isoforms.\n
          {n_nr_isoforms_g27} non redundant isoforms.
          \t{n_nr_pc_genes_g27} protein-coding genes.
          '''
    print(msg)

In [ ]:
get_final_trifid_stats(df_g27_predictions)

---

# 6. Interpreting our predictions <a class="anchor" id="interpret-the-predictions"></a>

## 6.1. Global interpretation <a class="anchor" id="global-interpretation"></a>

TRIFID uses Tree SHAP algorithms to explain the output of ensemble tree models.

In [ ]:
interpretation = TreeInterpretation(
    model = my_model,
    df = df_training_set, 
    features_col=df_training_set[features].columns,
    target_col='label',
    random_state=SEED,
    test_size=0.25)

In [ ]:
interpretation.shap.head(10)

In [ ]:
# df_merged_feature_importances = interpretation.merge_feature_importances
# df_merged_feature_importances.head()

## 6.2. Local interpretations <a class="anchor" id="local-interpretations"></a>

### 6.2.1. Local interpretation for set of gene isoforms <a class="anchor" id="local-interpretation-for-set-of-gene-isoforms"></a>

In [ ]:
interpretation.local_explanation(df_g27, 'PHACTR3')

In [ ]:
interpretation.local_explanation(df_g35, 'PHACTR3')

In [ ]:
interpretation.local_explanation(df_rs109, 'DCLK3')

### 6.2.2. Local interpretation for a single isoform <a class="anchor" id="local-interpretation-for-a-single-isoform"></a>

In [ ]:
interpretation.local_explanation(df_g35, sample='ENST00000651089').head(10)

In [ ]:
interpretation.local_explanation(df_rs109, sample='NM_006088').head(20)

---

# 7. Project contribution <a class="anchor" id="project-contribution"></a>

**Author information**:

Fernando Pozo

- [ORCID iD (0000-0001-7688-6045)](https://orcid.org/0000-0001-7688-6045)
- [GitLab (@fpozoc)](https://gitlab.com/fpozoc)
- [Twitter (@fpozoca)](https://twitter.com/fpozoca)